In [1]:
import beepy
import numpy as np
import time
import datetime
from dateutil.relativedelta import relativedelta
import tushare as ts
import pandas as pd
import matplotlib.pyplot as plt

# Set token here
pro = ts.pro_api('')

In [77]:
# Helper functions

def countdown_timer(x, now=datetime.datetime.now):
    target = now()
    one_second_later = datetime.timedelta(seconds=1)
    for remaining in range(x, 0, -1):
        target += one_second_later
        print(datetime.timedelta(seconds=remaining), 'remaining', end='\r')
        time.sleep((target - now()).total_seconds())
    print('\nTIMER ended')
    
    
def get_data(pro, t, startDate, endDate, fileds):
    try:
        df = pro.daily(ts_code=t, start_date=startDate, end_date=endDate, fields='ts_code,trade_date,high,low,close')
    except:
        df = get_data(pro, t, startDate, endDate, fileds)
        
    return df

def get_all_hist_data(pro, stockdata, startDate, endDate, fields, step, savecsv=1):
    [rows, cols] = stockdata.shape
    frames = []
    for k in range(0,int(np.ceil(rows/step))):
    
        countdown_timer(60)

        if(k == np.floor(rows/step)):
            tickers = stockdata['ts_code'][k*step:]
        else:
            tickers = stockdata['ts_code'][k*step:(k+1)*step]

        for t in tickers:
            df = get_data(pro, t, startDate, endDate, fields)
            print(t, end='\r')
            frames.append(df)
            if k == 7:
                if t is tickers[-1:].to_string(index=False).strip():
                    print("\nDone.")
                    beepy.beep(sound=1)
            else:
                if t is tickers[(k+1)*step - 1]:
                    print("\nDone.")
                    beepy.beep(sound=1)
                
    myStockData = pd.concat(frames, ignore_index=1)
    if savecsv == 1:
        myStockData.to_csv('myStockData.csv', encoding='utf-8-sig')
        
    return myStockData


def findStock(dataframe, trade_date, queries):
    
    m = re.match(pattern, trade_date)
    if m is not None:
        if m.group(1) == '>':
            df = dataframe.loc[dataframe['trade_date'] > m.group(2)]
        elif m.group(1) == '<':
            df = dataframe.loc[dataframe['trade_date'] < m.group(2)]
    else:
        print('Please input correct data')
        return []
    
    if 'high' in queries:
        mydf = df.loc[(df['high'] == queries['high'])]
    elif 'low' in queries:
        mydf = df.loc[(df['low'] == queries['low'])]

    candidates = np.unique(mydf['ts_code'])
    potentials = []
    for t in candidates:
        fData = mydf.loc[(mydf['ts_code']== t) & (mydf['close'] > queries['lower_bound']) & (mydf['close'] < queries['upper_bound'])]
        if(len(fData) != 0):
            potentials.append(fData)
            
    beepy.beep(sound=1)
    return pd.concat(potentials, ignore_index=1)

In [27]:
stockdata = pd.read_csv('stock_tickers.csv')

# Time range
endDate = '20200428'
t = datetime.datetime.strptime(endDate, '%Y%m%d')
t_startDate = t - relativedelta(years=2)
startDate = t_startDate.strftime('%Y%m%d')

# Pause after step to avoid ip banned
steps = 500

In [28]:
myStockData = get_all_hist_data(pro, stockdata, startDate, endDate, 'ts_code,trade_date,high,low,close', steps, savecsv=1)

8.0
0:00:01 remaining
TIMER ended

Start batch 0
002043.SZ
Done.
0:00:01 remaining
TIMER ended

Start batch 1
002549.SZ
Done.
0:00:01 remaining
TIMER ended

Start batch 2
300098.SZ
Done.
0:00:01 remaining
TIMER ended

Start batch 3
300606.SZ
Done.
0:00:01 remaining
TIMER ended

Start batch 4
600353.SH
Done.
0:00:01 remaining
TIMER ended

Start batch 5
600985.SH
Done.
0:00:01 remaining
TIMER ended

Start batch 6
603612.SH
Done.
0:00:01 remaining
TIMER ended

Start batch 7


In [75]:
# myStockData = pd.read_csv('myStockData.csv')
df = findStock(myStockData, '>20200425', {'low': 1.72, 'lower_bound':1.5, 'upper_bound':2})